# Tutorial: Private Deep Learning

Welcome to PySyft's introductory tutorial for privacy preserving deep learning. This series of notebooks is a step-by-step guide for you to get to know the new tools and techniques required for doing deep learning on secret data. 

## Why Take This Tutorial?

**1) A Competitive Career Advantage** - For the past 20 years, the digital revolution has made data more and more accessible in ever larger quantities as analog processes have become digitized. However, with new regulation such as [GDPR](https://eugdpr.org/), enterprises are under pressure to have less freedom with how they use - and more importantly how they analyze - personal information. **Bottom Line:** Data Scientists aren't going to have access to as much data with "old school" tools, but by learning the tools of Private Deep Learning, YOU can be ahead of this curve and have a competitive advantage in you career. 

**2) Entrepreneurial Opportunities** - There are a whole host of problems in society that Deep Learning can solve, but many of the most important haven't been explored because it would require access to incredibly sensitive information about people (consider using Deep Learning to help people with mental or relationship issues!). Thus, learning Private Deep Learning unlocks a whole host of new startup opportunities for you which were not previously available to others without these toolsets.

**3) Social Good** - Deep Learning can be used to solve a wide variety of problems in the real world, but Deep Learning on *personal information* is Deep Learning about people, *for people*. Learning how to do Deep Learning on data you don't own represents more than a career or entrepreneurial opportunity, it is the opportunity to help solve some of the most personal and important problems in people's lives - and to do it at scale.


## What do I need to know?

**1) Deep Learning with PyTorch** - if you don't just hop on over to [fast.ai](https://fast.ai), learn Deep Learning and PyTorch, and come back here.

## How do I get extra credit?

- Star PySyft on Github! - [https://github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)
- Make a Youtube video teaching this notebook!


... ok ... let's do this!

# Part 0: Setup

To begin, you'll need to make sure you have the right things installed. To do so, head on over to PySyft's readme and follow the setup instructions. TLDR for most folks is.

- Install Python 3.5 or higher
- Install PyTorch 0.3.1 (it MUST be this version)
- Clone PySyft (git clone https://github.com/OpenMined/PySyft.git)
- cd PySyft
- pip install -r requirements.txt
- python setup.py install
- python setup.py test

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
# Run this cell to see if things work
import syft as sy

sy.FloatTensor([1,2,3,4,5])


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

If this cell executed, then you're off to the races! Let's do this!

# Part 1: The Basic Tools of Private Data Science

So - the first question you may be wondering is - How in the world do we train a model on data we don't have access to? 

Well, the answer is surprisingly simple. If you're used to working in PyTorch, then you're used to working with torch.Tensor objects like these!

In [2]:
import torch
x = torch.FloatTensor([1,2,3,4,5])
y = x + x
print(y[0])

2.0


Obviously, using these super fancy (and powerful!) tensors is important, but also requires you to have the data on your local machine. This is where our journey begins. 

# Section 1.1 - Sending Tensors to Bob's Machine

Instead of Tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [3]:
bob = sy.VirtualWorker(id="bob")

For all intenstive purposes, Bob's machine is on another planet - perhaps on Mars! But, at the moment the machine is empty. Let's create some data so that we can send it to Bob and learn about pointers!

In [4]:
x = torch.FloatTensor([1,2,3,4,5])
y = torch.FloatTensor([1,1,1,1,1])

And now - let's send our tensors to Bob!!

In [5]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [6]:
x_ptr

FloatTensor[_PointerTensor - id:2887225416 owner:me loc:bob id@loc:63273344926]

BOOM! Now Bob has two tensors! Don't believe me? Have a look for youself!

In [7]:
bob._objects

{1055531132: [_LocalTensor - id:1055531132 owner:bob],
 63273344926: [_LocalTensor - id:63273344926 owner:bob]}

Now notice something. When we print our pointer...

In [8]:
x_ptr

FloatTensor[_PointerTensor - id:2887225416 owner:me loc:bob id@loc:63273344926]

We some useful metadata! First, ID of our tensor is 5648307076, that makes sense... it was allocated a random ID. However, there's also a few other pieces of metadata.


- loc: bob
- id@loc: 69591406818
- owner: me

Hopefully the naming of these attributes is quite intuitive. "loc" is short for "location" and it is a reference to the location that the pointer is pointing to! See?

In [9]:
x_ptr.location

<syft.core.workers.virtual.VirtualWorker id:bob>

In [10]:
bob

<syft.core.workers.virtual.VirtualWorker id:bob>

In [11]:
bob == x_ptr.location

True

The "id@loc" parameter is similar. It tells us the id that the Tensor object on Bob's machine has (the one that we're pointing to). See?

In [12]:
x_ptr.id_at_location

63273344926

In [13]:
bob._objects[x_ptr.id_at_location].parent


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

And finally - we have the third attribute "owner: me" which is very similar to ".location". However, instead of specifying where the pointer is pointing, it specifies the owner of the pointer itself, which is me. Fun fact, just like we had a VirtualWorker object for Bob, we (by default) always have one for us as well. This worker is automatically created when we "import syft"

In [14]:
me = sy.local_worker
me

<syft.core.workers.virtual.VirtualWorker id:me>

In [15]:
x_ptr.owner

<syft.core.workers.virtual.VirtualWorker id:me>

In [16]:
me == x_ptr.owner

True

And finally, just like we can call .send() on a tensor, we can call .get() on a pointer to a tensor to get it back!!!

In [17]:
x_ptr

FloatTensor[_PointerTensor - id:2887225416 owner:me loc:bob id@loc:63273344926]

In [18]:
x_ptr.get()


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [19]:
y_ptr

FloatTensor[_PointerTensor - id:2372370479 owner:me loc:bob id@loc:1055531132]

In [20]:
y_ptr.get()


 1
 1
 1
 1
 1
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [21]:
bob._objects

{}

And as you can see... Bob no longer has the tensors anymore!!! They've moved back to our machine!

# Section 1.2 - Using Tensor Pointers

So, sending and receiving tensors from Bob is great, but this is hardly Deep Learning! We want to be able to perform tensor _operations_ on remote tensors. Fortunately, tensor pointers make this quite easy!! You can just use poiners like you would normal tensors!

In [22]:
x = sy.FloatTensor([1,2,3,4,5]).send(bob)
y = sy.FloatTensor([1,1,1,1,1]).send(bob)

In [23]:
z = x + y

In [24]:
z

FloatTensor[_PointerTensor - id:6318014333 owner:me loc:bob id@loc:7313746027]

In [25]:
z.get()


 2
 3
 4
 5
 6
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

### Errors

However, if you try to do an operation between two tensors which aren't on the same machine, you'll get an error that looks like this!

In [26]:
x = sy.FloatTensor([1,2,3,4,5]).send(bob)
y = sy.FloatTensor([1,1,1,1,1])

In [28]:
z = y + x

TypeError: isinstance expected 2 arguments, got 1